# Práctica 3: Representación del conocimiento


El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 11__

__Nombres de los estudiantes: Alejandro Barrachina Argudo y Jaime Benedí Galdeano__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural).

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.

Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [3]:
%%wdsparql
SELECT ?item ?itemLabel
WHERE
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

/home/_alk/Documentos/uni/IA2/.venv/lib/python3.11/site-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,item,itemLabel
0,http://www.wikidata.org/entity/Q3569037,William Windsor
1,http://www.wikidata.org/entity/Q13050139,Q13050139
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I
7,http://www.wikidata.org/entity/Q117303721,Q117303721
8,http://www.wikidata.org/entity/Q117303807,Q117303807
9,http://www.wikidata.org/entity/Q117303856,Q117303856


### Consulta 1

[Stephen King (Q39829)](https://www.wikidata.org/wiki/Q39829) es un escritor estadounidense de novelas de terror, ficción sobrenatural, misterio, ciencia ficción y literatura fantástica.

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [4]:
%%wdsparql
SELECT ?birthTimeLabel ?birthPlaceLabel ?countryLabel
WHERE
{
  # King       fecha variable
  wd:Q39829 wdt:P569 ?birthTime.  
  # King      sitio nacimiento   variable
  wd:Q39829 wdt:P19 ?birthPlace.
  # sitio nacimiento pertenece a variable
  ?birthPlace wdt:P17 ?country.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,birthTimeLabel,birthPlaceLabel,countryLabel
0,1947-09-21T00:00:00Z,Portland,United States of America


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [5]:
%%wdsparql
SELECT ?professionLabel
WHERE
{
  # King      profesion variable
  wd:Q39829 wdt:P106 ?profession. 
  # Ciudad de nacimiento
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?professionLabel)

,professionLabel
0,actor
1,columnist
2,director
3,film director
4,journalist
5,novelist
6,science fiction writer
7,screenwriter
8,teacher
9,television producer


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [6]:
%%wdsparql
SELECT ?professionLabel
WHERE
{
  # King    profesion variable
  wd:Q39829 wdt:P106 ?profession. 
  # Profesion  subtipo de  Escritor
  ?profession wdt:P279+ wd:Q36180. # Sin poner el + hay relaciones que pueden no ser exploradas
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?professionLabel)

,professionLabel
0,columnist
1,novelist
2,science fiction writer
3,screenwriter
4,writer


### Consulta 4

King se ha casado una vez, con Tabitha Spruce, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento (si están disponibles). Los resultados deben aparecer ordenados cronológicamente.

In [7]:
%%wdsparql
SELECT ?childLabel ?birthDate
WHERE
{
  # King    descendencia variable
  wd:Q39829 wdt:P40 ?child.  # Nombre de stephen king
  OPTIONAL 
  {
    # hijo  fecha nacimiento variable
    ?child wdt:P569 ?birthDate.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?childLabel)

,childLabel,birthDate
0,Joe Hill,1972-06-04T00:00:00Z
1,Naomi King,None
2,Owen King,1977-02-21T00:00:00Z


### Consulta 5

King ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible) y la obra premiada (si está disponible). Los resultados se deben mostrar ordenados por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [8]:
%%wdsparql
SELECT ?fecha ?premioLabel ?obraLabel
WHERE
{
#   King       premios variable
    wd:Q39829 p:P166 ?nodo.
#   nodo  premios  variable
    ?nodo ps:P166 ?premio.
    OPTIONAL { 
        # nodo fecha  variable
        ?nodo pq:P585 ?fecha. 
        # nodo obra adjudicada variable
        ?nodo pq:P1686 ?obra.
    }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?fecha

/home/_alk/Documentos/uni/IA2/.venv/lib/python3.11/site-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,fecha,premioLabel,obraLabel
0,None,Shirley Jackson Award,None
1,None,Hammett Prize,None
2,None,O. Henry Award,None
3,None,Edgar Awards,None
4,None,National Medal of Arts,None
5,None,World Horror Convention Grand Master Award,None
6,None,Grand Prix de l'Imaginaire,None
7,1982-01-01T00:00:00Z,Hugo Award for Best Related Work,Danse Macabre
8,1996-01-01T00:00:00Z,Bram Stoker Award for Novel,The Green Mile
9,1997-01-01T00:00:00Z,Grand Prix de l'Imaginaire,Danse Macabre


### Consulta 6

King es sin duda un autor prolífico. ¿Cuántas obras ha escrito a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [9]:
%%wdsparql
SELECT (COUNT(?work) as ?Cuenta)
WHERE
{
  # variable instancia de obras literarias
  ?work wdt:P31 wd:Q7725634;
    # autor  King
    wdt:P50 wd:Q39829.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,Cuenta
0,226


### Consulta 7

Aunque King es mundialmente conocido por sus obras de ficción ([Q8253](https://www.wikidata.org/wiki/Q8253)), es también autor de algunas obras menos conocidas de no-ficción ([Q213051](https://www.wikidata.org/wiki/Q213051)). Queremos saber cuáles son estas obras de no-ficción (o cualquiera de sus subgéneros) y su tipo. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [10]:
%%wdsparql
SELECT DISTINCT?workLabel
WHERE
{
  # variable genero/subclase de no ficcion
  ?work wdt:P136*/wdt:P279* wd:Q213051;
  #  autor   King
    wdt:P50 wd:Q39829.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?workLabel)

/home/_alk/Documentos/uni/IA2/.venv/lib/python3.11/site-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,workLabel
0,Danse Macabre
1,Guns
2,Head Down
3,On Writing


### Consulta 8

De todas las obras que ha publicado, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [11]:
%%wdsparql
SELECT DISTINCT ?workLabel ?fecha
WHERE
{
  # variable instancia de obra literaria
  ?work wdt:P31 wd:Q7725634;
  #  autor   King
    wdt:P50 wd:Q39829;
    # fecha publicacion variable
    wdt:P571 ?fecha.
  FILTER("1969-12-31"^^xsd:dateTime <= ?fecha && ?fecha < "1980-01-01"^^xsd:dateTime).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,workLabel,fecha
0,Graveyard Shift,1970-01-01T00:00:00Z
1,The Mangler,1972-01-01T00:00:00Z
2,Suffer the Little Children,1972-01-01T00:00:00Z
3,Trucks,1973-01-01T00:00:00Z
4,The Boogeyman,1973-01-01T00:00:00Z
5,Night Surf,1974-01-01T00:00:00Z
6,Sometimes They Come Back,1974-01-01T00:00:00Z
7,Carrie,1974-01-01T00:00:00Z
8,The Lawnmower Man,1975-01-01T00:00:00Z
9,The Revenge of Lard Ass Hogan,1975-07-01T00:00:00Z


### Consulta 9

Wikidata contiene información actualizada que en ocasiones incluye futuros lanzamientos u obras planificadas. Vamos a dejar por un momento a King y vamos a considerar la obra de otro famoso autor, George R. R. Martin.

[George R. R. Martin (Q181677)](https://www.wikidata.org/wiki/Q181677) es un escritor de fantasía estadounidense conocido sobre todo por su saga "Canción de hielo y fuego". Martin tiene inconclusa la saga y queremos saber si hay alguna información sobre los libros pendientes. Estos serán sus obras sin fecha de publicación asociada. ¿Aparecen? Si lo hacen, ¿cuántas y cuáles son sus títulos? ¿Hay alguna obra pendiente que no sea de esa saga?

In [15]:
%%wdsparql
SELECT ?workLabel ?sagaLabel ?fecha
WHERE
{
  # variable instancia de obra literaria
  ?work wdt:P31 wd:Q7725634;
  # autor     Martin
    wdt:P50 wd:Q181677;
    OPTIONAL
    {
      # obra pertenece a saga
      ?work wdt:P179 ?saga.
    }
    OPTIONAL
    {
      # obra fecha lanzamiento variable
      ?work wdt:P577 ?fecha.
    }
    FILTER(!BOUND(?fecha)). # No tiene fecha de salida
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?sagaLabel) ASC(?nombre)

/home/_alk/Documentos/uni/IA2/.venv/lib/python3.11/site-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,workLabel,sagaLabel,fecha
0,Q3795208,None,NaN
1,Deuces Down,None,NaN
2,Marked Cards,None,NaN
3,Songs the Dead Men Sing,None,NaN
4,The Winds of Winter,A Song of Ice and Fire,NaN
5,Blood of the Dragon,A Song of Ice and Fire,NaN
6,Dead Man's Hand,Wild Cards,NaN


### Consulta 10

Volvemos de nuevo a King. De todas las obras que ha publicado (independientemente de su tipo y género), queremos recuperar los nombres y fechas de publicación de aquellas escritas en coautoría, así como los nombnres de estos coautores.

In [16]:
%%wdsparql
SELECT ?workLabel ?authorLabel ?fecha
WHERE
{
  # variable instancia de obra literaria
  ?work wdt:P31 wd:Q7725634;
  # autor    King
    wdt:P50 wd:Q39829.
  # obras autor variable
  ?work wdt:P50 ?author.
  OPTIONAL 
  {
    # obras  fecha lanzamiento variable 
    ?work wdt:P577 ?fecha
  }
  FILTER(?author != wd:Q39829) # autor distinto de King

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?workLabel

/home/_alk/Documentos/uni/IA2/.venv/lib/python3.11/site-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,workLabel,authorLabel,fecha
0,A Face in the Crowd,Stewart O'Nan,2014-03-17T00:00:00Z
1,Black House,Peter Straub,None
2,Gwendy's Button Box,Richard Chizmar,1999-05-16T00:00:00Z
3,Gwendy's Final Task,Richard Chizmar,2022-02-15T00:00:00Z
4,In the Tall Grass,Joe Hill,2012-08-01T00:00:00Z
5,Road Rage,Richard Matheson,2009-01-01T00:00:00Z
6,Sleeping Beauties,Owen King,2017-09-26T00:00:00Z
7,The Talisman,Peter Straub,1984-11-08T00:00:00Z


### Consulta 11

¿Con cuántos coautores diferentes ha trabajado King, independientemente del tipo y género de la obra?

In [17]:
%%wdsparql
SELECT DISTINCT (COUNT(?author) as ?Coautores)
WHERE
{
  # variable instancia de obra literaria
  ?work wdt:P31 wd:Q7725634;
   # autor   King
    wdt:P50 wd:Q39829.
  # obra autor   variable
  ?work wdt:P50 ?author.
  FILTER(?author != wd:Q39829) # autor distinto de King

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,Coautores
0,8


### Consulta 12

¿Qué otros personajes famosos nacieron el mismo año que King y tienen su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

In [18]:
%%wdsparql
SELECT ?famosoLabel ?kingBirthTimeLabel ?birthPlaceLabel ?countryLabel
WHERE
{
  # King    fecha nacimiento  variable
  wd:Q39829 wdt:P569 ?kingBirthTime.
  # Variable fecha nacimiento fecha nacimiento King
  ?famoso wdt:P569 ?kingBirthTime.
  # persona lugar nacimiento variable
  ?famoso wdt:P19 ?birthPlace.
  #lugar nacimiento Pais variable
  ?birthPlace wdt:P17 ?country.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,famosoLabel,kingBirthTimeLabel,birthPlaceLabel,countryLabel
0,Marek Valenta,1947-09-21T00:00:00Z,Kraków,Poland
1,Ed Nimmervoll,1947-09-21T00:00:00Z,Austria,Austria
2,Reggie Rucker,1947-09-21T00:00:00Z,"Washington, D.C.",United States of America
3,Nick Castle,1947-09-21T00:00:00Z,Los Angeles,United States of America
4,Rupert Hine,1947-09-21T00:00:00Z,London,United Kingdom
5,Nemezjusz Mateusz Pazio,1947-09-21T00:00:00Z,Warsaw,Poland
6,Levana Finkelstein,1947-09-21T00:00:00Z,Sofia,Bulgaria
7,Anatoly Alekseyev,1947-09-21T00:00:00Z,Saint Petersburg,Russia
8,Jean-Claude Anaf,1947-09-21T00:00:00Z,Grenoble,France
9,Partho Mukherjee,1947-09-21T00:00:00Z,Kolkata,India


### Consulta 13

¿Qué otros personajes famosos incluyen King en su nombre completo (name in native language, P1559)? Para cada uno de ellos muestra su nombre, fecha de nacimiento, ocupación y nacionalidad. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

In [19]:
%%wdsparql
SELECT DISTINCT ?name ?fecha ?ocupacion ?ocupacionLabel ?nacionalidadLabel
WHERE
{
  # persona nombre variable
    ?famoso wdt:P1559 ?name;
    # fecha nacimiento variable
    wdt:P569 ?fecha;
    # trabajo variable
    wdt:P106 ?ocupacion;
    # nacionalidad variable
    wdt:P27 ?nacionalidad.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  FILTER regex(?name, ".*(King).*", "i")  # incluye King en su nombre
}

ORDER BY ASC(?name) ASC(?fecha) ASC(?ocupacion)
# LIMIT 5



,name,fecha,ocupacion,ocupacionLabel,nacionalidadLabel
0,Tiong King Sing,1961-09-03T00:00:00Z,http://www.wikidata.org/entity/Q82955,politician,Malaysia
1,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q131512,farmer,Kingdom of Denmark
2,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q2405480,voice actor,Kingdom of Denmark
3,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q2526255,film director,Kingdom of Denmark
4,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q82955,politician,Kingdom of Denmark
5,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q28389,screenwriter,Kingdom of Denmark
6,Jens Okking,1939-12-18T00:00:00Z,http://www.wikidata.org/entity/Q33999,actor,Kingdom of Denmark
7,Alban Sumana Kingsford Bagbin,1957-09-24T00:00:00Z,http://www.wikidata.org/entity/Q808967,barrister,Ghana
8,Alban Sumana Kingsford Bagbin,1957-09-24T00:00:00Z,http://www.wikidata.org/entity/Q82955,politician,Ghana
9,Adelheid Winking-Nikolay,1944-08-13T00:00:00Z,http://www.wikidata.org/entity/Q82955,politician,Germany


### Consulta 14

¿Cuántos asertos hay sobre King en Wikidata? Ten en cuenta que puede aparecer tanto como sujeto como objeto de una tripleta.

In [20]:
%%wdsparql
SELECT (COUNT(*) AS ?asertos) #cuento todas las filas
WHERE 
{
  {
    SELECT ?suj ?verb
    WHERE
    {
      ?suj ?verb wd:Q39829. #cosas que tienen como valor de una cualidad a King
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
  }
  UNION 
  {
    SELECT ?verb ?predic
    WHERE
    {
      wd:Q396 ?verb ?predic. # cosas que son valor de alguna cualidad de King
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
  } 
}

,asertos
0,2600


__Fecha de las consultas__: 13-02-2024